In [11]:
import os
import time
import random

import pandas as pd
import numpy as np

# from pyspark.sql import SparkSession
# from pyspark import SparkContext
# from pyspark.mllib.regression import LabeledPoint #라벨을 표시하기 위해 사용
# from pyspark.mllib.feature import HashingTF #단어를 숫자로 변환하고 빈도수를 매핑하기 위해 사용
# from pyspark.mllib.classification import LogisticRegressionWithSGD #SGD 즉 확률론적경사하강법을 적용한 로지스틱 회귀분석을 위해 사용

from pyspark.python.pyspark.shell import sqlContext
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator


Exception: Java gateway process exited before sending its port number

In [6]:
os.listdir()

['train_set_9.csv',
 'train_set_8.csv',
 'train_set_3.csv',
 'train_set_2.csv',
 'train_set_0.csv',
 'train_set_1.csv',
 'train_set_5.csv',
 'train_set_4.csv',
 'train_set_6.csv',
 'train_set_7.csv']

In [9]:
iris = sqlContext.read.format("com.databricks.spark.csv")\
                    .options(header='true', inferSchema='true')\
                    .load('../data/*.csv')\
                    .repatition(2).cache()

NameError: name 'sqlContext' is not defined

In [ ]:
#-*-coding:utf-8-*-



#print(iris.show())

stridx = StringIndexer(inputCol='name', outputCol="name_idx")
#encode = OneHotEncoder(inputCol = "name_idx", outputCol="name_encode") #라벨은 데이터가 아니기 때문에
#print(encode)
#stages=[encode]

inputs = ["SepalLength","SepalWidth","PetalLength","PetalWidth"] 
va = VectorAssembler(inputCols=inputs, outputCol="features")




pipeline= Pipeline(stages=[va, stridx])
pipelineModel = pipeline.fit( iris)
dataDF = pipelineModel.tranform(iris).drop("SepalLength","SepalWidth","PetalLength","PetalWidth","Name")
print(dataDF.show())


(train, test) = dataDF.randomSplit([0.7,0.3], seed=0)
lr = LogisticRegression(labelCol="name_idx", featuresCol='features',maxIter=10)
lrModel = lr.fit(train)
predict = lrModel.transform(test)
#predict.show()

evaluator = BinaryClassificationEvaluator(labelCol='name_idx', rawPredictionCol="rawPredictionCol")
print(evaluator.getMetricName())   #AUC
print(evaluator.evaluate( predict))

#교차검증으로 정확한 값을 찾기 
paramGrid = ( ParamGridBuilder().addGrid(lr.regParam, [0.01,0.1])).addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]).addGrid(lr.maxIter, [1,5,10]).build()




cv=CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator= evaluator, numFolds=5)
cvModel = cv.fit(train)
predict= cvModel.transform(test)
print(cvModel.bestModel.getMaIter)
print(cvModel.bestModel.getRegParam())
print(cvModel.bestModel.getElasticNetParam())






